In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data

In [ ]:
df=pd.read_csv("../input/donorsprediction/Raw_Data_for_train_test.csv")

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

# Find missing values

In [ ]:
df.isnull().sum()

In [ ]:
df.columns[df.isnull().any()]

create a new column called DONATION_TYPE

In [ ]:
# create a list of our conditions
conditions = [
    (df['TARGET_D'] >= 50),
    (df['TARGET_D'] >= 20) & (df['TARGET_D'] < 50),
    (df['TARGET_D'] >= 13) & (df['TARGET_D'] < 20),
    (df['TARGET_D'] >= 10) & (df['TARGET_D'] < 13),
    (df['TARGET_D'] < 10)
    ]

# create a list of the values we want to assign for each condition
values = ['A', 'B', 'C', 'D','E']

# create a new column and use np.select to assign values to it using our lists as arguments
df['DONATION_TYPE'] = np.select(conditions, values)

# display updated DataFrame
df.head()

In [ ]:
df

# Missing values

In [ ]:
df.isnull().sum()

In [ ]:
df['TARGET_D']=df['TARGET_D'].fillna(df['TARGET_D'].mean())

In [ ]:
# create a list of our conditions
conditions = [
    (df['TARGET_D'] >= 50),
    (df['TARGET_D'] >= 20) & (df['TARGET_D'] < 50),
    (df['TARGET_D'] >= 13) & (df['TARGET_D'] < 20),
    (df['TARGET_D'] >= 10) & (df['TARGET_D'] < 13),
    (df['TARGET_D'] < 10)
    ]

# create a list of the values we want to assign for each condition
values = ['A', 'B', 'C', 'D','E']

# create a new column and use np.select to assign values to it using our lists as arguments
df['DONATION_TYPE'] = np.select(conditions, values)

# display updated DataFrame
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#target d contains a lot of null values ,so delete this column
df=df.drop(['TARGET_D'],axis=1)

In [ ]:
df=df.drop(['TARGET_B'],axis=1)

In [ ]:
df['DONOR_AGE']=df['DONOR_AGE'].fillna(df['DONOR_AGE'].mean())

In [ ]:
df['DONOR_AGE']=df['DONOR_AGE'].astype('int64')

In [ ]:
df['INCOME_GROUP']=df['INCOME_GROUP'].fillna(df['INCOME_GROUP'].mode()[0])

In [ ]:
df['INCOME_GROUP']=df['INCOME_GROUP'].astype('int64')

In [ ]:
df['WEALTH_RATING']=df['WEALTH_RATING'].fillna(df['WEALTH_RATING'].mode()[0])

In [ ]:
df['WEALTH_RATING']=df['WEALTH_RATING'].astype('int64')

In [ ]:
df=df.dropna()

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df

In [ ]:
len(df.DONATION_TYPE.unique())

In [ ]:
df['DONATION_TYPE'].value_counts()

# make label encoder to categorical coulmns

In [ ]:
from sklearn.preprocessing import LabelEncoder

labeled_df = df.copy()

label_encoder = LabelEncoder().fit (df['DONATION_TYPE'])

labeled_species = label_encoder.transform(df['DONATION_TYPE'])

In [ ]:
classes = list(label_encoder.classes_)  
classes

In [ ]:
labeled_df

drop  categorical col.

In [ ]:
labeled_df=labeled_df.drop('URBANICITY',axis=1)
labeled_df=labeled_df.drop('SES', axis=1)
labeled_df=labeled_df.drop('HOME_OWNER', axis=1)
labeled_df=labeled_df.drop('DONOR_GENDER', axis=1)
labeled_df=labeled_df.drop('OVERLAY_SOURCE', axis=1)
labeled_df=labeled_df.drop('CLUSTER_CODE', axis=1)
labeled_df=labeled_df.drop('RECENCY_STATUS_96NK', axis=1)

In [ ]:
labeled_df

# Define X and y

In [ ]:
X=labeled_df.drop('DONATION_TYPE', axis=1)
y=labeled_df.DONATION_TYPE

In [ ]:
X

In [ ]:
y

# Build model

Spilt data into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train

In [ ]:
X_train.dtypes

#  Hypter-parameter tuning

In [ ]:
''''parameters = {
    'n_estimators': list(range(100, 1001, 100)), 
    'learning_rate': [l / 100 for l in range(5, 50, 10)], 
    'max_depth': list(range(6, 20, 10))
}
parameters'''

In [ ]:
para = list(range(100, 1001, 100))
print(para)

# Define GridSearchCV for hypter-parameter tuning

In [ ]:
''''from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

gsearch = GridSearchCV(estimator=XGBClassifier(random_state=1,objective='multi:softprob'),
                       param_grid = parameters, 
                       scoring='neg_log_loss',
                       n_jobs=4,cv=5, verbose=1)'''

# Random Forest classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
results = {}
for n in para:
    print('para=', n)
    model = RandomForestClassifier(n_estimators=n)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accu = accuracy_score(y_true=y_test, y_pred=preds)
    f1 = f1_score(y_true=y_test, y_pred=preds, average='micro')
    print(classification_report(y_true=y_test, y_pred=preds))
    print('--------------------------')
    results[n] = f1

In [ ]:
import matplotlib.pylab as plt
# sorted by key, return a list of tuples
lists = sorted(results.items()) 
p, a = zip(*lists) # unpack a list of pairs into two tuples
plt.plot(p, a)
plt.show()

In [ ]:
best_para = max(results, key=results.get)
print('best para', best_para)
print('value', results[best_para])

# Predict the test data

In [ ]:
test_df = pd.read_csv('../input/donorsprediction/Predict_donor.csv')
test_df.head()

In [ ]:
test_df.columns[test_df.isnull().any()]

In [ ]:
test_df['DONOR_AGE']=test_df['DONOR_AGE'].fillna(test_df['DONOR_AGE'].mean())

In [ ]:
test_df['DONOR_AGE']=test_df['DONOR_AGE'].astype('int64')

In [ ]:
test_df['INCOME_GROUP']=test_df['INCOME_GROUP'].fillna(test_df['INCOME_GROUP'].mode()[0])

In [ ]:
test_df['INCOME_GROUP']=test_df['INCOME_GROUP'].astype('int64')

In [ ]:
test_df['WEALTH_RATING']=test_df['WEALTH_RATING'].fillna(test_df['WEALTH_RATING'].mode()[0])

In [ ]:
test_df['WEALTH_RATING']=test_df['WEALTH_RATING'].astype('int64')

In [ ]:
test_df=test_df.dropna()

In [ ]:
test_df.columns[test_df.isnull().any()]

In [ ]:
test_df.dtypes

In [ ]:
test_df

In [ ]:
test_df=test_df.drop('URBANICITY',axis=1)
test_df=test_df.drop('SES', axis=1)
test_df=test_df.drop('HOME_OWNER', axis=1)
test_df=test_df.drop('DONOR_GENDER', axis=1)
test_df=test_df.drop('OVERLAY_SOURCE', axis=1)
test_df=test_df.drop('CLUSTER_CODE', axis=1)
test_df=test_df.drop('RECENCY_STATUS_96NK', axis=1)

In [ ]:
test_df.dtypes

In [ ]:
test_df.columns

In [ ]:
test_df

In [ ]:
test_df.shape

In [ ]:
X_train.shape

In [ ]:
X_train.dtypes

In [ ]:
Target = model.predict(test_df)
Target

In [ ]:
PREDICTED_df = pd.DataFrame()
PREDICTED_df['DONATION_TYPE'] = Target
PREDICTED_df['CONTROL_NUMBER'] = test_df['CONTROL_NUMBER']
PREDICTED_df.head()